In [24]:
from arcgis.features import GeoAccessor
routes_path = r'D:\projects\geoai-retail\data\interim\interim.gdb\ClosestFacility1\CFRoutes1'

In [25]:
df = GeoAccessor.from_featureclass(routes_path)
print(len(df.index))
df.head()

10


,ObjectID,FacilityID,FacilityRank,Name,IncidentCurbApproach,FacilityCurbApproach,IncidentID,StartTime,EndTime,StartTimeUTC,EndTimeUTC,Total_Minutes,Total_TravelTime,Total_Miles,Total_Kilometers,Total_TimeAt1KPH,Total_WalkTime,Total_TruckMinutes,Total_TruckTravelTime,SHAPE
0,1,1781,1,637497520 - 410510013023,2,1,649,None,None,None,None,None,1.791511,None,0.730751,None,None,None,None,"{""hasM"": true, ""paths"": [[[-13651821.6007, 570..."
1,2,1800,2,666724190 - 410510013023,2,1,649,None,None,None,None,None,1.797261,None,0.668861,None,None,None,None,"{""hasM"": true, ""paths"": [[[-13651532.17, 57013..."
2,3,1802,3,724992959 - 410510013023,1,1,649,None,None,None,None,None,2.021772,None,0.688978,None,None,None,None,"{""hasM"": true, ""paths"": [[[-13651109.156, 5702..."
3,4,1776,4,402794004 - 410510013023,2,2,649,None,None,None,None,None,2.131916,None,0.926105,None,None,None,None,"{""hasM"": true, ""paths"": [[[-13652099.8994, 570..."
4,5,1799,5,439386809 - 410510013023,2,1,649,None,None,None,None,None,2.159315,None,0.784228,None,None,None,None,"{""hasM"": true, ""paths"": [[[-13651376.0585, 570..."


In [26]:
pair_lst = df.Name.apply(lambda val: val.split(' - '))

In [27]:
df['destination_id'] = [pair[1] for pair in pair_lst]
df['origin_id'] = [pair[0] for pair in pair_lst]
df['proximity_miles'] = df['Total_Kilometers'] * 0.621371
df.head()

,ObjectID,FacilityID,FacilityRank,Name,IncidentCurbApproach,FacilityCurbApproach,IncidentID,StartTime,EndTime,StartTimeUTC,...,Total_Miles,Total_Kilometers,Total_TimeAt1KPH,Total_WalkTime,Total_TruckMinutes,Total_TruckTravelTime,SHAPE,destination_id,origin_id,proximity_miles
0,1,1781,1,637497520 - 410510013023,2,1,649,None,None,None,...,None,0.730751,None,None,None,None,"{""hasM"": true, ""paths"": [[[-13651821.6007, 570...",410510013023,637497520,0.454068
1,2,1800,2,666724190 - 410510013023,2,1,649,None,None,None,...,None,0.668861,None,None,None,None,"{""hasM"": true, ""paths"": [[[-13651532.17, 57013...",410510013023,666724190,0.415611
2,3,1802,3,724992959 - 410510013023,1,1,649,None,None,None,...,None,0.688978,None,None,None,None,"{""hasM"": true, ""paths"": [[[-13651109.156, 5702...",410510013023,724992959,0.428111
3,4,1776,4,402794004 - 410510013023,2,2,649,None,None,None,...,None,0.926105,None,None,None,None,"{""hasM"": true, ""paths"": [[[-13652099.8994, 570...",410510013023,402794004,0.575455
4,5,1799,5,439386809 - 410510013023,2,1,649,None,None,None,...,None,0.784228,None,None,None,None,"{""hasM"": true, ""paths"": [[[-13651376.0585, 570...",410510013023,439386809,0.487297


In [28]:
df_od = df[['origin_id', 'destination_id', 'proximity_miles', 'Total_TravelTime', 'SHAPE']].copy()
df_od.columns = ['origin_id', 'destination_id', 'proximity_miles', 'proximity_drive_time', 'SHAPE']
df_od

,origin_id,destination_id,proximity_miles,proximity_drive_time,SHAPE
0,637497520,410510013023,0.454068,1.791511,"{""hasM"": true, ""paths"": [[[-13651821.6007, 570..."
1,666724190,410510013023,0.415611,1.797261,"{""hasM"": true, ""paths"": [[[-13651532.17, 57013..."
2,724992959,410510013023,0.428111,2.021772,"{""hasM"": true, ""paths"": [[[-13651109.156, 5702..."
3,402794004,410510013023,0.575455,2.131916,"{""hasM"": true, ""paths"": [[[-13652099.8994, 570..."
4,439386809,410510013023,0.487297,2.159315,"{""hasM"": true, ""paths"": [[[-13651376.0585, 570..."
5,417755879,410510013023,0.468911,2.211764,"{""hasM"": true, ""paths"": [[[-13650808.5934, 570..."
6,724497190,410510013023,0.483467,2.259268,"{""hasM"": true, ""paths"": [[[-13650886.5169, 570..."
7,415522508,410510013023,0.575721,2.659192,"{""hasM"": true, ""paths"": [[[-13650675.0099, 570..."
8,404971434,410510013023,0.603943,2.806587,"{""hasM"": true, ""paths"": [[[-13650649.1326, 570..."
9,421989050,410510013023,0.716267,2.920494,"{""hasM"": true, ""paths"": [[[-13652144.4272, 570..."


In [29]:
sdf_od = df_od.set_geometry('SHAPE')
sdf_od.sample(5)

,origin_id,destination_id,proximity_miles,proximity_drive_time,SHAPE
1,666724190,410510013023,0.415611,1.797261,"{""hasM"": true, ""paths"": [[[-13651532.17, 57013..."
3,402794004,410510013023,0.575455,2.131916,"{""hasM"": true, ""paths"": [[[-13652099.8994, 570..."
0,637497520,410510013023,0.454068,1.791511,"{""hasM"": true, ""paths"": [[[-13651821.6007, 570..."
7,415522508,410510013023,0.575721,2.659192,"{""hasM"": true, ""paths"": [[[-13650675.0099, 570..."
8,404971434,410510013023,0.603943,2.806587,"{""hasM"": true, ""paths"": [[[-13650649.1326, 570..."


In [30]:
df_coffee = GeoAccessor.from_featureclass('D:\\projects\\geoai-retail\\data\\raw\\raw.gdb\\coffee')
df_coffee.sample(5)

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REMOVE,open_status,SHAPE
512,513,479827917,STARBUCKS,NE WEIDLER ST,PORTLAND,OR,Oregon,97232,1851,72251505,...,4,,3,PointAddress,M,100.0,INFOGROUP,0,None,"{""x"": -122.63499999995327, ""y"": 45.53319999958..."
548,549,415522509,PORTLAND COFFEE BREAK,SE STARK ST,PORTLAND,OR,Oregon,97233,2153,72251505,...,,,2,PointAddress,M,100.0,INFOGROUP,0,None,"{""x"": -122.51430000015557, ""y"": 45.51960000014..."
259,260,721916382,STARBUCKS,SW 6TH AVE,PORTLAND,OR,Oregon,97204,1102,72251505,...,4,,2,PointAddress,M,100.0,INFOGROUP,0,None,"{""x"": -122.68020000027519, ""y"": 45.51679999996..."
160,161,637497850,STARBUCKS,SW PACIFIC HWY,TIGARD,OR,Oregon,97223,8845,72251505,...,4,,2,PointAddress,M,100.0,INFOGROUP,0,None,"{""x"": -122.75209999966336, ""y"": 45.44109999976..."
579,580,243189032,STARBUCKS,NE 112TH AVE,VANCOUVER,WA,Washington,98684,4283,72251505,...,4,,2,PointAddress,M,100.0,INFOGROUP,0,None,"{""x"": -122.55670000029824, ""y"": 45.64129999957..."


In [31]:
df_coffee = df_coffee[['LOCNUM', 'CONAME']].copy()
df_coffee.set_index('LOCNUM', inplace=True)
df_coffee.sample(5)

,CONAME
LOCNUM,
637352923,STARBUCKS
706070672,DUTCH BROTHERS COFFEE
388731242,FLOYD'S COFFEE SHOP
709126719,DUTCH BROTHERS COFFEE
372249466,COFFEE RUSH


In [32]:
sdf_od = sdf_od.join(df_coffee, on='origin_id')
sdf_od

,origin_id,destination_id,proximity_miles,proximity_drive_time,SHAPE,CONAME
0,637497520,410510013023,0.454068,1.791511,"{""hasM"": true, ""paths"": [[[-13651821.6007, 570...",STARBUCKS
1,666724190,410510013023,0.415611,1.797261,"{""hasM"": true, ""paths"": [[[-13651532.17, 57013...",FLYING CAT COFFEE CO
2,724992959,410510013023,0.428111,2.021772,"{""hasM"": true, ""paths"": [[[-13651109.156, 5702...",PEAKS COFFEE PDX
3,402794004,410510013023,0.575455,2.131916,"{""hasM"": true, ""paths"": [[[-13652099.8994, 570...",COAVA COFFEE ROASTERS
4,439386809,410510013023,0.487297,2.159315,"{""hasM"": true, ""paths"": [[[-13651376.0585, 570...",TOV
5,417755879,410510013023,0.468911,2.211764,"{""hasM"": true, ""paths"": [[[-13650808.5934, 570...",COFFEE DIVISION
6,724497190,410510013023,0.483467,2.259268,"{""hasM"": true, ""paths"": [[[-13650886.5169, 570...",SEVEN VIRTUES COFFEE ROASTERS
7,415522508,410510013023,0.575721,2.659192,"{""hasM"": true, ""paths"": [[[-13650675.0099, 570...",UNCLE GARY'S COFFEE EMPORIUM
8,404971434,410510013023,0.603943,2.806587,"{""hasM"": true, ""paths"": [[[-13650649.1326, 570...",STARBUCKS
9,421989050,410510013023,0.716267,2.920494,"{""hasM"": true, ""paths"": [[[-13652144.4272, 570...",K & F SELECT FINE COFFEES


In [40]:
sdf_od.spatial.to_featureclass('D:\\projects\\geoai-retail\\data\\interim\\interim.gdb\\example_trips')

'D:\\projects\\geoai-retail\\data\\interim\\interim.gdb\\example_trips'

In [37]:
from arcgis.gis import GIS
webmap = GIS().map('Portland, OR')
sdf_od.spatial.plot(webmap)

True

In [38]:
webmap

MapView(layout=Layout(height='400px', width='100%'))